In [1]:
# import libraries (non-cryptogaphic)
import random # to generate phone numbers
import pandas as pd


# import libraries (cryptographic)
import cryptography.hazmat.primitives.asymmetric.dh as dh
from cryptography.fernet import Fernet
import hashlib
import sympy
import secrets

# Generate phone numbers and store them in phone_numbers.csv file

In [2]:
random.seed(10) # to ensure same phone numbers generated every time

In [3]:
# class to generate phone numbers for grab and gojek
class PhoneNumberGenerator:
    def __call__(self, count):
        phone_numbers = random.sample(range(80000000,100000000), count)
        return phone_numbers 

    
# class to store numbers to csv
class PhoneNumberStorageManager:
    def __init__(self):
        self.filename = "phone_numbers.csv"
    def __call__(self, gojek_phone_numbers, grab_phone_numbers):
        d = {"gojek": gojek_phone_numbers, 
            "grab": grab_phone_numbers}
        df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()])) # create dataframe
        df.to_csv(self.filename, index = False) # store values to file "phone_numbers.csv"

### Edit cell below to change phone number count

In [4]:
# set phone number count
gojek_phone_number_count = 61 # inclusive of phone numbers in common with grab
grab_phone_number_count = 91 # inclusive of phone numbers in common with gojek
common_phone_number_count = 10

In [5]:
# instantiate required classes
phone_number_generator = PhoneNumberGenerator()
phone_number_storage_manager = PhoneNumberStorageManager()

# generate phone numbers
phone_numbers = phone_number_generator(gojek_phone_number_count+grab_phone_number_count-common_phone_number_count)
common_phone_numbers = phone_numbers[0:common_phone_number_count]
gojek_phone_numbers = phone_numbers[0:gojek_phone_number_count]
grab_phone_numbers = common_phone_numbers + phone_numbers[gojek_phone_number_count:] 

#shuffle phone number lists
random.shuffle(gojek_phone_numbers)
random.shuffle(grab_phone_numbers)

# write phone numbers to csv file
phone_number_storage_manager(gojek_phone_numbers, grab_phone_numbers)

# Define classes for the necessary for the algorithm

In [6]:
# class to generate numbers required for psi
class NumberGenerator:
    
    def generate_public_parameters(self, size):
        # method to generate p, q and factors of p-1
        p = self.generate_safe_prime(size)
        print(f"p is prime: {sympy.ntheory.isprime(p)}")
        length_of_p = len(bin(p)[2:]) # should be 1024
        print(f"Length of prime modulus, p: {length_of_p}.\nNote: Should be {size}.")
        q = (p-1)//2
        print(f"q is prime: {sympy.ntheory.isprime(q)}")
        factors_pminus1 = [1, 2, q] # since q is prime, 2q only has these 3 factors excluding itself
                                    # 2q = p-1
                                    # factors of p-1 required to compute order of generators (lagrange theorem)
        return p, factors_pminus1

    def generate_safe_prime(self, size):
        # method to generate safe prime for p
        candidate = dh.generate_parameters(2, size).parameter_numbers().p # generate 1024-bit prime number
        while True:
            # the method used from the cryptography already generates a safe prime, this portion is merely to double confirm
            is_safe_prime = sympy.ntheory.isprime((candidate-1)//2) # if safe prime, (candidate-1)/2 is prime
            if (is_safe_prime):
                break
            else:
                candidate = dh.generate_parameters(2, size).parameter_numbers().p
                print(candidate)
 
        return candidate
    
    
    def generate_random_number(self, size):
        # method to generate client's secret
        return secrets.randbits(size)
            
# class to inspect values          
class NumberInspector:
    
    def check_is_primitive_generator(self, candidate, factors_divisorminus1, divisor): # note: factors should be the factors of divisor-1
        
        # apply lagrange theorem
        for possible_order in factors_divisorminus1: # check congruence for all factors (factors is exclusive of p-1 itself)
            result = pow(candidate, possible_order, divisor) # fast modular exponentiation implemented in pow(x,y,z)
                                                             # python integers have arbitrary precisions, 
                                                             # no overflow would occur if operations done in pure python
            if (result == 1):
                return False # if candidate exponent any of the factors (1,2,q) congruent to 1modp, candidate is not a primitive
                             # generator
        return True # order of candidate == p-1, therefore candidate is a primitive generator

    
class StorageManager:
    
    def store_data(self, filename, data):
        df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe
        df.to_csv(filename, index=False) # store data to file with filename

                


    
        

# Create psi client class

In [7]:
# client class (both grab and gojek are clients communicating directly with each other)
class Client:
    def __init__(self, name, other_party_name, private_key_size, phone_numbers, p, factors_pminus1, fernet_key):
        
        self.number_inspector = NumberInspector()
        self.number_generator = NumberGenerator()
        
        # for asymmetric encryption
        self.private_key = self.number_generator.generate_random_number(private_key_size)
        self.my_set = phone_numbers
        self.p = p # prime modulus
        self.factors_pminus1 = factors_pminus1 # to calculate order of hashed phone numbers, 
                                               # since algorithm requires them to be primitive generators
        
        # for symmetric encryption
        self.fernet_key = fernet_key
        self.f = Fernet(fernet_key)
        
        # values to track for psi algorithm
        self.my_hashed_set = None # h(x)
        self.my_self_encrypted_set = None # (h(x)^(my_secret))modp
        self.my_encrypted_set = None # (h(x)^(my_secret)(other_party_secret))modp
        self.other_party_encrypted_set = None # (h(y)^(my_secret)(other_party_secret))modp
        self.common_values = None # common phone numbers
        
        
        # create datafile for communication with another party
        
        # content to store in file
        # only need to share self_encrypted_values and other_party_encrypted_values
        # common_values to ensure both calculate the same intersection
        self.my_dict = {
            'my_self_encrypted_set': None, 
            'other_party_encrypted_set': None,
            'common_values': None
        }
        
        # filenames
        self.name = name
        self.filename = name + "_data_v1.1.csv"
        self.other_party_name = other_party_name
        self.other_party_filename = other_party_name + "_data_v1.1.csv"
        
        # create file
        self.storage_manager = StorageManager()
        self.storage_manager.store_data(self.filename, self.my_dict)

    def hash_to_primitive_root_modulo_p(self, element): 
        # method to hash phone numbers to primitive root modulo p i.e. primitive generator

        endian = "big"
        element = element.to_bytes(4, endian)
        hash_hex = hashlib.sha256(element).hexdigest() # sha3_256
        hash_int = int(hash_hex, 16)
        while True:
            # repeatedly hash until primitive root modulo p is obtained
            is_primitive_generator = self.number_inspector.check_is_primitive_generator(
                hash_int, self.factors_pminus1, self.p
            )
            if (is_primitive_generator):
                break
            else:
                hash_int = hash_int.to_bytes(32, endian)
                hash_hex = hashlib.sha256(hash_int).hexdigest()
                hash_int = int(hash_hex, 16)
                
        return hash_int
    
    def modular_exponentation(self, element):
        # compute (element^(private_key))modp

        return pow(element, self.private_key, self.p)
    
    def hash_set(self):
        # hash all phone numbers in my set to primitive root modulo p, one by one

        self.my_hashed_set = []
        
        for element in self.my_set:
            hashed_value = self.hash_to_primitive_root_modulo_p(element)
            self.my_hashed_set.append(hashed_value)
            
    def encrypt_set(self, is_other_party):
        # encrypt all elements in a given set using private_key, one by one
        
        # two scenarios to consider
        # one: encrypt set sent by the other party
        if (is_other_party):
            decrypted_other_party_set = self.receive_data("my_self_encrypted_set")
            other_party_set_int = []
            for element_string in decrypted_other_party_set:
                other_party_set_int.append(int(element_string))
            set_to_encrypt = other_party_set_int
        # two: encrypt my own set
        else:
            set_to_encrypt = self.my_hashed_set
        
        # encrypt values in given set, one by one
        encrypted_values = []
        for element in set_to_encrypt:
            encrypted_value = self.modular_exponentation(element)
            encrypted_values.append(encrypted_value)
            
        # assign the encrypted set to the correct variable
        # update csv file used for communication
        if (is_other_party):
            self.other_party_encrypted_set = encrypted_values
            self.send_data(encrypted_values, "other_party_encrypted_set")
        else:
            self.my_self_encrypted_set = encrypted_values
            self.send_data(encrypted_values, "my_self_encrypted_set")
                    
        
    def get_intersection(self):
        # get intersection
        
        my_encrypted_set = self.receive_data("other_party_encrypted_set") # read my encrypted set from the other party's file
        my_encrypted_set_int = []
        
        # convert read values to integer
        for element in my_encrypted_set:
            my_encrypted_set_int.append(int(element))
            
        # assign to correct variable
        self.my_encrypted_set = my_encrypted_set_int
        
        # get intersection
        encrypted_common_values = set(self.my_encrypted_set).intersection(self.other_party_encrypted_set)
        index_of_common_values = []
        
        # find the index of the elements in the intersection in my_encrypted_set
        for element in encrypted_common_values:
            index_of_common_values.append(self.my_encrypted_set.index(element))
            
        self.common_values = []
        
        # find the values in my own set corresponding to the index of the elements in the intersection
        for index in index_of_common_values:
            self.common_values.append(self.my_set[index])
            
        # update csv file for communication
        self.send_data(self.common_values, "common_values")
    
    def encrypt_data(self, plaintext):
        # encrypt data with Fernet
        
        endian = "big"
        element = plaintext.to_bytes(128, endian) # 1024 bits == 128 bytes
        cipher_text = self.f.encrypt(element)
        return cipher_text
    
    def decrypt_data(self, ciphertext):
        # decrypt data encrypted by Fernet
        
        ciphertext_bytes = ciphertext.encode('utf-8')[2:-1] # convert from string back to bytes
        endian = "big"
        element_in_bytes = self.f.decrypt(ciphertext_bytes)
        plaintext = int.from_bytes(element_in_bytes, endian)
        return plaintext

    def send_data(self, data_to_send, column_name):
        # send data means writing to file. encrypt data with Fernet
        
        # encrypt data
        encrypted_data_to_send = []
        for element in data_to_send:
            encrypted_element = self.encrypt_data(element)
            encrypted_data_to_send.append(encrypted_element)
            
        # send data
        self.my_dict[column_name] = encrypted_data_to_send
        self.storage_manager.store_data(self.filename, self.my_dict)
        
        
    def receive_data(self, column_name):
        # receive data means reading from file (my file). decrypt data encrypted by Fernet
        
        # receive data
        encrypted_data = self.get_other_party_data()[column_name].to_list()
        
        # decrypt data
        decrypted_data = []
        for element in encrypted_data:
            if type(element) is float: # remove NaN
                continue
            decrypted_element = self.decrypt_data(element)
            decrypted_data.append(decrypted_element)
        print(f"number of elements in {self.other_party_name};{column_name}: {len(decrypted_data)}.")
        return decrypted_data
    
    def get_my_data(self):
        # read my file as dataframe (other party's file)
        
        return pd.read_csv(self.filename)
            
    def get_other_party_data(self):
        # read other party's file as dataframe
        
        return pd.read_csv(self.other_party_filename)
    


# Initialize context

### Edit cell below to change key size

In [8]:
# assign pre-determined variables for psi
key_size = 1024 # both private keys and large prime

In [9]:
# create key for symmetric key cryptography
fernet_key = Fernet.generate_key()

# create public parameters required for psi
number_generator = NumberGenerator()
p, factors_pminus1 = number_generator.generate_public_parameters(key_size)

# create clients
grab = Client("grab", "gojek", key_size, grab_phone_numbers, p, factors_pminus1, fernet_key)
gojek = Client("gojek", "grab", key_size, gojek_phone_numbers, p, factors_pminus1, fernet_key)



p is prime: True
Length of prime modulus, p: 1024.
Note: Should be 1024.
q is prime: True


C:\Users\jiaji\AppData\Local\Temp/ipykernel_12120/964370003.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


# Get intersection

## Step 1: Hash phone numbers

In [10]:
# clients hash their own set
grab.hash_set()
gojek.hash_set()

### Clients' status after step 1 (value of variables in client)
Note: hashed set is not stored in the file as it is not meant to be shared with the other party hence, files' status after step 1 is not shown

In [11]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set
0,7501383186321213515512277498775635912732518619...,8093135284524341492510691949857707487260359145...
1,9113933134158773335900585734938644471316560047...,5930688136427591374346485570270456146366904370...
2,4984484178152826219285777261886375116635680274...,3003360056476023280396172869223514448918734055...
3,4271190063387899149690355220761209149002412790...,1155906544719417202618665601321621894146534318...
4,4320682999770379726688099447684191745405506617...,1057775043639589044287875970966910956491296265...
...,...,...
86,NaN,2625832542840555517519669698958112696515497351...
87,NaN,3683371775181761031500661295222456997304332645...
88,NaN,3062635494707074836246422469968877214990302591...
89,NaN,5011913842513548422134267800545138946266586949...


## Step 2: encrypt hashed set with own private key

In [12]:
# clients self encrypt hashed set
grab.encrypt_set(False) # set is_other_party to false to encrypt own hashed set
gojek.encrypt_set(False)

C:\Users\jiaji\AppData\Local\Temp/ipykernel_12120/964370003.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


### Clients' status after step 2 (value of variables in client)

In [13]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set
0,7501383186321213515512277498775635912732518619...,8093135284524341492510691949857707487260359145...,9910304707833415250984425287609586985813278055...,5666626654020178812671443389671926031275349123...
1,9113933134158773335900585734938644471316560047...,5930688136427591374346485570270456146366904370...,5765446841639957447608677739993533620375894219...,5187258947147492369255194001448988917667995286...
2,4984484178152826219285777261886375116635680274...,3003360056476023280396172869223514448918734055...,1224371722707901761151529575316754610185371514...,6466381875277869408182983097475544524498264069...
3,4271190063387899149690355220761209149002412790...,1155906544719417202618665601321621894146534318...,7886184086596377318796893017784493188916915522...,7272939346155048237827743342285916988456039287...
4,4320682999770379726688099447684191745405506617...,1057775043639589044287875970966910956491296265...,6686948265916342657999192777717527662153166272...,9345833300158422865482437460824401810882061546...
...,...,...,...,...
86,NaN,2625832542840555517519669698958112696515497351...,NaN,1438676334049701538860573960408365691399257535...
87,NaN,3683371775181761031500661295222456997304332645...,NaN,7234513617622807752336679122426735452307748693...
88,NaN,3062635494707074836246422469968877214990302591...,NaN,5305690256068250410582625868537659872459552538...
89,NaN,5011913842513548422134267800545138946266586949...,NaN,9459021928140896736874726703310374786658463447...


### Files' status after step 2 (value of variables in file - clients' variables encrypted with Fernet)

In [14]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek


gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhonb0goC418bmZFj_YYWVD5WfdEmbJ1LBywdZ...,NaN,NaN
1,b'gAAAAABhonb0YpLarxoYWwzZGgHrjTcpg1uWum1H1AXn...,NaN,NaN
2,b'gAAAAABhonb0rNqu1JOk0OVrANJayWO9rxi9XxYmjykm...,NaN,NaN
3,b'gAAAAABhonb0zFxhXmpj8LbIbkvBAJzHljNeDjwvVTXW...,NaN,NaN
4,b'gAAAAABhonb0nMussEAiTLO6m07ba0ov_r6J8xom8mHj...,NaN,NaN
...,...,...,...
56,b'gAAAAABhonb0eH0FUo5LseX89i2XoUG_qDqx812IX3hv...,NaN,NaN
57,b'gAAAAABhonb0nhnlf8j2D-WYXXugK9xQmqQVBL8-FRbq...,NaN,NaN
58,b'gAAAAABhonb05dEZPS4cQAd6di9iLgDuFOxsJIBdiUc1...,NaN,NaN
59,b'gAAAAABhonb0zW8oHFw6nZiIHiIUqlUdS3ZiKWZejT4C...,NaN,NaN


In [15]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhonbzPT5Hm2Oa-WVFlVR659bNV0l94yHHl3sw...,NaN,NaN
1,b'gAAAAABhonbzO3Y-mO1wxTRsZNkBOLoBLssi-2ZkZILk...,NaN,NaN
2,b'gAAAAABhonbz5wY8q0o_q9adBWJ1tS4iitGTqrWB1hmA...,NaN,NaN
3,b'gAAAAABhonbz81hB82T3FPUL95uzn5L7fxrTitrb1XfK...,NaN,NaN
4,b'gAAAAABhonbzbJC9itZflQDYQrD7camiVwX4tDvlQfRh...,NaN,NaN
...,...,...,...
86,b'gAAAAABhonbzTBGbbiTtywbHvnMoKEz7cjHe0U6ovw5W...,NaN,NaN
87,b'gAAAAABhonbzT4ORGYsUJDVKS5ASUr1lb07WniLiqU6w...,NaN,NaN
88,b'gAAAAABhonbztljuBPvX0-P2ukC3Zqd2b1CJTNLgyni6...,NaN,NaN
89,b'gAAAAABhonbzzy4Q5exx76agorGd5Mr1J5SlAlG-0IMn...,NaN,NaN


## Step 3: encrypt other party's self-encrypted set with own private key


In [16]:
# clients encrypt other party's self encrypted set
grab.encrypt_set(True) # set is_other_party to true
gojek.encrypt_set(True)

number of elements in gojek;my_self_encrypted_set: 61.


C:\Users\jiaji\AppData\Local\Temp/ipykernel_12120/964370003.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


number of elements in grab;my_self_encrypted_set: 91.


### Clients' status after step 3

In [17]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set,
    "gojek encrypted set": grab.other_party_encrypted_set,
    "grab encrypted set": gojek.other_party_encrypted_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set,gojek encrypted set,grab encrypted set
0,7501383186321213515512277498775635912732518619...,8093135284524341492510691949857707487260359145...,9910304707833415250984425287609586985813278055...,5666626654020178812671443389671926031275349123...,6352366672556279296920463028468833774589733507...,1057819772953314423667617303874118488505277740...
1,9113933134158773335900585734938644471316560047...,5930688136427591374346485570270456146366904370...,5765446841639957447608677739993533620375894219...,5187258947147492369255194001448988917667995286...,9033519499344170325394592319790660335462143918...,9122685345642926689437012603201955220490788673...
2,4984484178152826219285777261886375116635680274...,3003360056476023280396172869223514448918734055...,1224371722707901761151529575316754610185371514...,6466381875277869408182983097475544524498264069...,3533125105187177931411510296063640803871859701...,1200386628390108985891104258325837854381191047...
3,4271190063387899149690355220761209149002412790...,1155906544719417202618665601321621894146534318...,7886184086596377318796893017784493188916915522...,7272939346155048237827743342285916988456039287...,2810769475772777665446834015002632467501585115...,3724337461545563906537076833093342222907365792...
4,4320682999770379726688099447684191745405506617...,1057775043639589044287875970966910956491296265...,6686948265916342657999192777717527662153166272...,9345833300158422865482437460824401810882061546...,4068941005969238115735041866104156315824261127...,8666355264071959211654292569819016672178739271...
...,...,...,...,...,...,...
86,NaN,2625832542840555517519669698958112696515497351...,NaN,1438676334049701538860573960408365691399257535...,NaN,2267947346924896318341019776528185704249610979...
87,NaN,3683371775181761031500661295222456997304332645...,NaN,7234513617622807752336679122426735452307748693...,NaN,1990121297611932662152353943931519058187273818...
88,NaN,3062635494707074836246422469968877214990302591...,NaN,5305690256068250410582625868537659872459552538...,NaN,2366453823811964772257636982048536394326516183...
89,NaN,5011913842513548422134267800545138946266586949...,NaN,9459021928140896736874726703310374786658463447...,NaN,1683759380518557286776997285274255124184925430...


### Files' status after step 3

In [18]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek



gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhonb0goC418bmZFj_YYWVD5WfdEmbJ1LBywdZ...,b'gAAAAABhonb3ja-1BReX-IWy9Iu3fGERFKYg0bLd0M65...,NaN
1,b'gAAAAABhonb0YpLarxoYWwzZGgHrjTcpg1uWum1H1AXn...,b'gAAAAABhonb368xHb8ouB3i3yWhCvZvlya8OqFs8uEtt...,NaN
2,b'gAAAAABhonb0rNqu1JOk0OVrANJayWO9rxi9XxYmjykm...,b'gAAAAABhonb3R9LG3ggQy_GUIGLY41KKFbEiYwdC7_Eq...,NaN
3,b'gAAAAABhonb0zFxhXmpj8LbIbkvBAJzHljNeDjwvVTXW...,b'gAAAAABhonb3ud3y8G8wVLsYmxdqVV3yUPx1aRb3oAYb...,NaN
4,b'gAAAAABhonb0nMussEAiTLO6m07ba0ov_r6J8xom8mHj...,b'gAAAAABhonb3M5xjqIbwZvJBemMl0dnfzTn-zEg6Ao93...,NaN
...,...,...,...
86,NaN,b'gAAAAABhonb3YJSddxcscJ-PFDVAOuQYxiZc7RKA05g5...,NaN
87,NaN,b'gAAAAABhonb32jMvbGoNEzw3-Qf2-2s3jqm3lgloxErA...,NaN
88,NaN,b'gAAAAABhonb3LlKf2qrVV9arhysb4VkQ65MP6jLdoKT6...,NaN
89,NaN,b'gAAAAABhonb3tK-lMukhc-D-vVw26Lmzjmmwn2ZFMfXU...,NaN


In [19]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhonbzPT5Hm2Oa-WVFlVR659bNV0l94yHHl3sw...,b'gAAAAABhonb14qGLIfjMAmVvI3RHSKSMQobWl7i1Hx1H...,NaN
1,b'gAAAAABhonbzO3Y-mO1wxTRsZNkBOLoBLssi-2ZkZILk...,b'gAAAAABhonb1dOizodWpHZzXgoc-ZvP1E5xE-uOVJUOz...,NaN
2,b'gAAAAABhonbz5wY8q0o_q9adBWJ1tS4iitGTqrWB1hmA...,b'gAAAAABhonb1oJGW9nxIS95Ar-toL2kZfeokbTvGDHf_...,NaN
3,b'gAAAAABhonbz81hB82T3FPUL95uzn5L7fxrTitrb1XfK...,b'gAAAAABhonb1_QPYvW2IpuoUts_9gwiUOui3wshKcPLG...,NaN
4,b'gAAAAABhonbzbJC9itZflQDYQrD7camiVwX4tDvlQfRh...,b'gAAAAABhonb1ZCYOnvmhmstK7rlR_uAUPKPNbVnkDL1F...,NaN
...,...,...,...
86,b'gAAAAABhonbzTBGbbiTtywbHvnMoKEz7cjHe0U6ovw5W...,NaN,NaN
87,b'gAAAAABhonbzT4ORGYsUJDVKS5ASUr1lb07WniLiqU6w...,NaN,NaN
88,b'gAAAAABhonbztljuBPvX0-P2ukC3Zqd2b1CJTNLgyni6...,NaN,NaN
89,b'gAAAAABhonbzzy4Q5exx76agorGd5Mr1J5SlAlG-0IMn...,NaN,NaN


## Step 4: find intersection


In [20]:
# clients find intersection
grab.get_intersection()
gojek.get_intersection()

number of elements in gojek;other_party_encrypted_set: 91.
number of elements in grab;other_party_encrypted_set: 61.


### Clients' status after step 4:

In [21]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set,
    "gojek encrypted set": grab.other_party_encrypted_set,
    "grab encrypted set": gojek.other_party_encrypted_set,
    "gojek found intersection": gojek.common_values,
    "grab found intersection:": grab.common_values}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set,gojek encrypted set,grab encrypted set,gojek found intersection,grab found intersection:
0,7501383186321213515512277498775635912732518619...,8093135284524341492510691949857707487260359145...,9910304707833415250984425287609586985813278055...,5666626654020178812671443389671926031275349123...,6352366672556279296920463028468833774589733507...,1057819772953314423667617303874118488505277740...,99173089.0,99173089.0
1,9113933134158773335900585734938644471316560047...,5930688136427591374346485570270456146366904370...,5765446841639957447608677739993533620375894219...,5187258947147492369255194001448988917667995286...,9033519499344170325394592319790660335462143918...,9122685345642926689437012603201955220490788673...,94391128.0,94391128.0
2,4984484178152826219285777261886375116635680274...,3003360056476023280396172869223514448918734055...,1224371722707901761151529575316754610185371514...,6466381875277869408182983097475544524498264069...,3533125105187177931411510296063640803871859701...,1200386628390108985891104258325837854381191047...,80497694.0,80497694.0
3,4271190063387899149690355220761209149002412790...,1155906544719417202618665601321621894146534318...,7886184086596377318796893017784493188916915522...,7272939346155048237827743342285916988456039287...,2810769475772777665446834015002632467501585115...,3724337461545563906537076833093342222907365792...,81093373.0,81093373.0
4,4320682999770379726688099447684191745405506617...,1057775043639589044287875970966910956491296265...,6686948265916342657999192777717527662153166272...,9345833300158422865482437460824401810882061546...,4068941005969238115735041866104156315824261127...,8666355264071959211654292569819016672178739271...,86915509.0,86915509.0
...,...,...,...,...,...,...,...,...
86,NaN,2625832542840555517519669698958112696515497351...,NaN,1438676334049701538860573960408365691399257535...,NaN,2267947346924896318341019776528185704249610979...,NaN,NaN
87,NaN,3683371775181761031500661295222456997304332645...,NaN,7234513617622807752336679122426735452307748693...,NaN,1990121297611932662152353943931519058187273818...,NaN,NaN
88,NaN,3062635494707074836246422469968877214990302591...,NaN,5305690256068250410582625868537659872459552538...,NaN,2366453823811964772257636982048536394326516183...,NaN,NaN
89,NaN,5011913842513548422134267800545138946266586949...,NaN,9459021928140896736874726703310374786658463447...,NaN,1683759380518557286776997285274255124184925430...,NaN,NaN


Note: Last 2 columns, unlike the rest of the columns, do not have a one-to-one mapping with other values belonging to the same row i.e. values in the last 2 columns do not have any relation to the other values in the same row as it.

### Files' status after step 4

In [22]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek


gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhonb0goC418bmZFj_YYWVD5WfdEmbJ1LBywdZ...,b'gAAAAABhonb3ja-1BReX-IWy9Iu3fGERFKYg0bLd0M65...,b'gAAAAABhonb30mrrAiJaD7nwxYFiyy9yPEOVsmPneMiv...
1,b'gAAAAABhonb0YpLarxoYWwzZGgHrjTcpg1uWum1H1AXn...,b'gAAAAABhonb368xHb8ouB3i3yWhCvZvlya8OqFs8uEtt...,b'gAAAAABhonb30jhpk4ksp-dIOuQTplZ__2q-d-qdzSU0...
2,b'gAAAAABhonb0rNqu1JOk0OVrANJayWO9rxi9XxYmjykm...,b'gAAAAABhonb3R9LG3ggQy_GUIGLY41KKFbEiYwdC7_Eq...,b'gAAAAABhonb3qBqXlYBi9Cpkd3Uwy0eRoYj1hHAcVJS6...
3,b'gAAAAABhonb0zFxhXmpj8LbIbkvBAJzHljNeDjwvVTXW...,b'gAAAAABhonb3ud3y8G8wVLsYmxdqVV3yUPx1aRb3oAYb...,b'gAAAAABhonb3H804lrKRcZnvSBFp04wC9IF0aANZnE7t...
4,b'gAAAAABhonb0nMussEAiTLO6m07ba0ov_r6J8xom8mHj...,b'gAAAAABhonb3M5xjqIbwZvJBemMl0dnfzTn-zEg6Ao93...,b'gAAAAABhonb3I8QszMgL_7a69jaTkDTpqhu4hmGJbDLD...
...,...,...,...
86,NaN,b'gAAAAABhonb3YJSddxcscJ-PFDVAOuQYxiZc7RKA05g5...,NaN
87,NaN,b'gAAAAABhonb32jMvbGoNEzw3-Qf2-2s3jqm3lgloxErA...,NaN
88,NaN,b'gAAAAABhonb3LlKf2qrVV9arhysb4VkQ65MP6jLdoKT6...,NaN
89,NaN,b'gAAAAABhonb3tK-lMukhc-D-vVw26Lmzjmmwn2ZFMfXU...,NaN


In [23]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhonbzPT5Hm2Oa-WVFlVR659bNV0l94yHHl3sw...,b'gAAAAABhonb14qGLIfjMAmVvI3RHSKSMQobWl7i1Hx1H...,b'gAAAAABhonb39Pa9U-2xeof52INq9GQ42tqv-0G1pBH0...
1,b'gAAAAABhonbzO3Y-mO1wxTRsZNkBOLoBLssi-2ZkZILk...,b'gAAAAABhonb1dOizodWpHZzXgoc-ZvP1E5xE-uOVJUOz...,b'gAAAAABhonb3Lc3u0IUh8ry2P-mz-EQqIxrFKaADY--D...
2,b'gAAAAABhonbz5wY8q0o_q9adBWJ1tS4iitGTqrWB1hmA...,b'gAAAAABhonb1oJGW9nxIS95Ar-toL2kZfeokbTvGDHf_...,b'gAAAAABhonb3qVOuPp5uVSgBjRE_Av4FZUCfaLzPHos9...
3,b'gAAAAABhonbz81hB82T3FPUL95uzn5L7fxrTitrb1XfK...,b'gAAAAABhonb1_QPYvW2IpuoUts_9gwiUOui3wshKcPLG...,b'gAAAAABhonb3tgIegXKyILx6qbG_HwQrPC9G_kNGxYAG...
4,b'gAAAAABhonbzbJC9itZflQDYQrD7camiVwX4tDvlQfRh...,b'gAAAAABhonb1ZCYOnvmhmstK7rlR_uAUPKPNbVnkDL1F...,b'gAAAAABhonb33gcgVdqG93CVzkfVAZ2w-htanqcsSBIn...
...,...,...,...
86,b'gAAAAABhonbzTBGbbiTtywbHvnMoKEz7cjHe0U6ovw5W...,NaN,NaN
87,b'gAAAAABhonbzT4ORGYsUJDVKS5ASUr1lb07WniLiqU6w...,NaN,NaN
88,b'gAAAAABhonbztljuBPvX0-P2ukC3Zqd2b1CJTNLgyni6...,NaN,NaN
89,b'gAAAAABhonbzzy4Q5exx76agorGd5Mr1J5SlAlG-0IMn...,NaN,NaN


# Check results

In [24]:
# get intersection found by the two parties
gojek_found_intersection = gojek.common_values
grab_found_intersection = grab.common_values


# sort numbers for easier comparison
gojek_found_intersection.sort()
grab_found_intersection.sort()
common_phone_numbers.sort()

# summarize them in a dataframe
d = {"actual": common_phone_numbers,
    "gojek": gojek_found_intersection,
    "grab": grab_found_intersection}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,actual,gojek,grab
0,80497694,80497694,80497694
1,81093373,81093373,81093373
2,86915509,86915509,86915509
3,89312048,89312048,89312048
4,94391128,94391128,94391128
5,95521626,95521626,95521626
6,96192082,96192082,96192082
7,96485172,96485172,96485172
8,99173089,99173089,99173089
9,99397525,99397525,99397525
